In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.model_selection

In [3]:
# Load the data
Xtrain = pd.read_csv('prepped_data/Xtrain.csv')
Xtest = pd.read_csv('prepped_data/Xtest.csv')
Ytest = pd.read_csv('prepped_data/Ytest.csv')
Ytrain = pd.read_csv('prepped_data/Ytrain.csv')
Xtrain_KNN = pd.read_csv('prepped_data/Xtrain_KNN.csv')
Xtest_KNN = pd.read_csv('prepped_data/Xtest_KNN.csv')

In [5]:
Xtrain.head()

,-4.025844893918861800e-01,-1.129822917661571502e+00,-1.363445721603272931e+00,-5.371761607391217774e-01,7.368661477636514467e-01,6.810471411349976467e-01,-1.334217355163681784e+00,-1.141483584549927155e+00,-1.957934751801259754e+00,8.918569882410334326e-01,...,-7.914537422070756501e-01,-4.826377174475534049e-01,-3.849697471031540696e-02,-5.821894183063817119e-01,-7.840436799237101573e-01,-5.501119809707601949e-01,-1.162221266410563558e+00,-7.082208037604180717e-01,5.479742555242571589e-01,6.810471411349976467e-01.1
0,-0.402584,-1.129823,-1.363446,-0.537176,0.736866,0.267494,-1.334217,-1.141484,-1.957935,0.891857,...,-0.877057,-0.629784,-0.063324,-0.605996,-0.655887,-0.547750,-1.142499,-0.445193,-0.408144,0.267494
1,-1.164473,-1.240314,-1.357197,-0.537176,0.736866,0.681047,-1.334217,-1.141484,-1.957935,0.891857,...,-0.791454,-0.482638,-0.038497,-0.582189,-0.784044,-0.550112,-1.162221,-0.708221,0.547974,0.681047
2,0.608831,-1.172271,-1.475839,-0.537176,0.736866,0.267494,-1.334217,-1.141484,-1.957935,0.891857,...,-0.877057,-0.629784,-0.063324,-0.605996,-0.655887,-0.547750,-1.142499,-0.445193,-0.408144,0.267494
3,0.469096,-1.606308,-2.006224,-0.537176,0.736866,0.267494,-1.334217,-1.141484,-1.957935,0.891857,...,-0.877057,-0.629784,-0.063324,-0.605996,-0.655887,-0.547750,-1.142499,-0.445193,-0.408144,0.267494
4,-0.432528,-1.518714,-2.118348,-0.537176,0.736866,0.267494,-1.334217,-1.141484,-1.957935,0.891857,...,-0.877057,-0.629784,-0.063324,-0.605996,-0.655887,-0.547750,-1.142499,-0.445193,-0.408144,0.267494


In [4]:
#Performing a time series split on the training data to split it into 5 folds for cross validation
tscv = sklearn.model_selection.TimeSeriesSplit(n_splits=5,Xtrain)




#sklearn.model_selection.TimeSeriesSplit(n_splits=5, *, max_train_size=None, test_size=None, gap=0)

SyntaxError: positional argument follows keyword argument (3994386528.py, line 2)